In [14]:
import PyPDF2
import autogen
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager

import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# Configuration for the language model
llm_config = {"model": "gpt-4o"}

# Define the agents
scope_agent = autogen.AssistantAgent(
    name="Scope and Change Management Risks Agent",
    llm_config=llm_config,
    system_message="""Scope and Change Management Risks Agent. Analyze the contract for risks related to scope clarity, change order procedures, and scope creep. Provide detailed findings based on your analysis.""",
)

manager_agent = autogen.AssistantAgent(
    name="Contract Analysis Manager",
    llm_config=llm_config,
    system_message="""Contract Analysis Manager. Coordinate the analysis of the contract by multiple agents, each focusing on a specific dimension of risk. Summarize the findings and provide a comprehensive risk analysis report.""",
)

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin..",
    code_execution_config=False,
    human_input_mode= "NEVER"
)

In [ ]:
def extract_text_from_pdf(pdf_file):
    pdf_text = ""
    with open(pdf_file, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            pdf_text += page.extract_text()
    return pdf_text

pdf_text = extract_text_from_pdf('/Users/yoginishanth/Documents/Python/DatabaseAgent/data/sample.pdf' )
print(pdf_text)


In [20]:

# Sequentially initiate the chat and print messages from each agent
def run_sequential_chat():
    user_proxy.initiate_chat(
        manager_agent,
        context_message={
            "role": "Admin",
            "content": f"Analyze the following contract for various risks:\n\n{pdf_text}"
        }
    )
    print("Admin initiated the chat.")
    
    for agent in groupchat.agents:
        if agent != user_proxy:
            response = agent.generate_reply([{"role": "Admin", "content": f"Analyze the following contract for various risks:\n\n{pdf_text}"}])
            print(f"{agent.name} responded with: {response['content']}")

run_sequential_chat()

KeyboardInterrupt: Interrupted by user

In [ ]:

groupchat = autogen.GroupChat(
    agents=[user_proxy, manager_agent, scope_agent], messages=[], max_round=1
)
for agent in groupchat.agents:
    agent.reset()
manager_agent= autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)
user_proxy.initiate_chat( manager_agent,
context_message = {
        "role": "Admin",
        "content": f"Analyze the following contract for various risks:\n\n{pdf_text}"
    }
)

In [17]:
#user_proxy.initiate_chat(
#    manager_agent,
#    context_message = {
#        "role": "Admin",
#        "content": f"Analyze the following contract for various risks:\n\n{pdf_text}"
#    }
#,
#)